# Export Demo
Demonstrates reading an Avro dataset + Avro | Parquet Cleanlab Columns, joins them, and saves result to CSV

In [1]:
import os
import pathlib

import pyspark.sql
import pyspark.sql.functions

import sys; sys.path.insert(0, "../src/")
from random_cleanlab_columns import generate_random_ood_columns, generate_random_cleanlab_columns
from sample_data import DATA_DIR, fetch_dataset

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.12:3.3.2 pyspark-shell'

spark = pyspark.sql.SparkSession.builder.appName(
    "dataset_view_demo"
).getOrCreate()

23/02/28 23:13:16 WARN Utils: Your hostname, Ryans-MacBook-Pro-3.local resolves to a loopback address: 127.0.0.1; using 192.168.0.4 instead (on interface en0)
23/02/28 23:13:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/ryansingman/.pyenv/versions/3.10.4/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/ryansingman/.ivy2/cache
The jars for the packages stored in: /Users/ryansingman/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e9fa030d-ee83-41fe-8845-8a6467705deb;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.3.2 in central
	found org.tukaani#xz;1.9 in central
	found org.spark-project.spark#unused;1.0.0 in central
:: resolution report :: resolve 130ms :: artifacts dl 4ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.3.2 from central in [default]
	org.spark-project.spark#unused;1.0.0 from central in [default]
	org.tukaani#xz;1.9 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      d

23/02/28 23:13:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/28 23:13:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/28 23:13:17 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# download dataset to use
DATASET_TO_USE = "Tweets-1M.csv"
DATASET_PATH = fetch_dataset(DATASET_TO_USE)

In [4]:
# load dataset to dataframe
dataset_df = spark.read.option("header", True).option("escape", '"').format("csv").load(str(DATASET_PATH))

# save dataset to Avro
AVRO_DATASET_PATH = DATASET_PATH.with_suffix(".avro")
dataset_df.write.format("avro").save(str(AVRO_DATASET_PATH), mode="overwrite")

+--------------------+-----------------+----------------------------+--------------+-------------------------+--------------+----------------------+---------------+-------------------+-------------+--------------------+-----------+--------------------+--------------------+--------------------+
|            tweet_id|airline_sentiment|airline_sentiment_confidence|negativereason|negativereason_confidence|       airline|airline_sentiment_gold|           name|negativereason_gold|retweet_count|                text|tweet_coord|       tweet_created|      tweet_location|       user_timezone|
+--------------------+-----------------+----------------------------+--------------+-------------------------+--------------+----------------------+---------------+-------------------+-------------+--------------------+-----------+--------------------+--------------------+--------------------+
|                   0|          neutral|                         1.0|          null|                     null|Virgi

Generate Cleanlab columns and save to Avro + Parquet

In [7]:
avro_filename_template = str(AVRO_DATASET_PATH).replace(".avro", "-{filetype}.avro")

def _create_filename(filetype: str, format: str) -> str:
    return str(AVRO_DATASET_PATH).replace(".avro", f"-{filetype}.{format}")

def save_df(df_table: pyspark.sql.DataFrame, filetype: str, format: str):
    outfile = _create_filename(filetype, format)

    df_table.write.format("avro").save(outfile, mode="overwrite")

In [8]:
# load spark table from avro
dataset_table = spark.read.format("avro").load(str(AVRO_DATASET_PATH))

id_column = "tweet_id"
label_column = "airline_sentiment"

# generate randomized cleanlab columns
cl_cols = generate_random_cleanlab_columns(dataset_table.alias("cl_cols_df"), id_column=id_column, label_column=label_column)

# generate randomized OOD columns
ood_cols = generate_random_ood_columns(dataset_table.alias("ood_cols_df"), id_column=id_column)

# save to avro and parquet
for format in ("avro", "parquet"):
    save_df(cl_cols, "cl_cols", format)
    save_df(ood_cols, "ood_cols", format)
